# Vision

- GPT-4o with image inpus

- GPT-4o와 GPT-4 Turbo 모두 비전 기능을 갖추고 있습니다. 즉, 모델이 이미지를 촬영하고 그에 대한 질문에 답할 수 있습니다

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

이미지는 두 가지 주요 방법, 즉 이미지에 대한 링크를 전달하거나 요청에 base64로 인코딩된 이미지를 직접 전달하는 방법으로 모델에서 사용할 수 있습니다. 

In [2]:
from openai import OpenAI

client = OpenAI()

### 이미지의 내용 설명
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

In [3]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "이 이미지의 내용을 설명해줘. 한국어로 설명해줘."},
        {
          "type": "image_url",
          "image_url": {
            "url": url,
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

response.to_dict()

{'id': 'chatcmpl-A6CfcFNMHJy49zvOML4oUaKZ5kQhP',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '이 이미지는 넓은 들판을 가로지르는 나무로 된 산책로를 보여줍니다. 들판은 푸른 풀과 다양한 식물들로 가득차 있으며, 산책로는 수평선까지 이어져 있습니다. 하늘은 맑고 푸르며, 몇 개의 작은 구름들이 흩어져 있는 모습을 보입니다. 전체적으로 평화롭고 자연적인 풍경을 담고 있습니다.',
    'role': 'assistant',
    'refusal': None}}],
 'created': 1726042036,
 'model': 'gpt-4o-2024-05-13',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_992d1ea92d',
 'usage': {'completion_tokens': 98,
  'prompt_tokens': 1127,
  'total_tokens': 1225}}

In [4]:
print(response.choices[0].message.content)

이 이미지는 넓은 들판을 가로지르는 나무로 된 산책로를 보여줍니다. 들판은 푸른 풀과 다양한 식물들로 가득차 있으며, 산책로는 수평선까지 이어져 있습니다. 하늘은 맑고 푸르며, 몇 개의 작은 구름들이 흩어져 있는 모습을 보입니다. 전체적으로 평화롭고 자연적인 풍경을 담고 있습니다.


### Base 64로 인코딩된 이미지 업로드
로컬에 이미지 또는 이미지 세트가 있는 경우 이를 Base 64 인코딩 형식으로 모델에 전달할 수 있습니다. 다음은 이에 대한 실제 예입니다.

In [13]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/냉장고내용물.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "이 이미지의 내용을 한국어로 설명해줘."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-A6CiRXoKSRG5ArH1rjNvf8LlF76UM', 'object': 'chat.completion', 'created': 1726042211, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '이 이미지는 냉장고 내부를 보여주고 있습니다. 냉장고에는 다양한 신선한 과일과 채소, 음료수, 그리고 기타 식품들이 깔끔하게 정리되어 있습니다. 윗쪽 선반에는 딸기나 오렌지 같은 과일과 함께 물병들이 보입니다. 중간 선반에는 양배추, 파프리카, 가지, 상추 등 다양한 채소들이 놓여 있습니다. 아래쪽 서랍에는 당근, 파슬리 등의 채소가 보입니다. 냉장고 문쪽에는 우유, 주스 등 다양한 음료수와 허브가 정리되어 있습니다. 전체적으로 신선하고 건강한 식재료들로 가득 차 있는 모습입니다.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 443, 'completion_tokens': 161, 'total_tokens': 604}, 'system_fingerprint': 'fp_992d1ea92d'}


In [14]:
res = response.json()
res['choices'][0]['message']['content']

'이 이미지는 냉장고 내부를 보여주고 있습니다. 냉장고에는 다양한 신선한 과일과 채소, 음료수, 그리고 기타 식품들이 깔끔하게 정리되어 있습니다. 윗쪽 선반에는 딸기나 오렌지 같은 과일과 함께 물병들이 보입니다. 중간 선반에는 양배추, 파프리카, 가지, 상추 등 다양한 채소들이 놓여 있습니다. 아래쪽 서랍에는 당근, 파슬리 등의 채소가 보입니다. 냉장고 문쪽에는 우유, 주스 등 다양한 음료수와 허브가 정리되어 있습니다. 전체적으로 신선하고 건강한 식재료들로 가득 차 있는 모습입니다.'

### 다중 이미지 입력
Chat Completions API는 base64로 인코딩된 형식이나 이미지 URL로 여러 이미지 입력을 가져와 처리할 수 있습니다. 모델은 각 이미지를 처리하고 모든 이미지의 정보를 사용하여 질문에 답합니다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

<img src="https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg" width=300px />

In [15]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "이 이미지에는 무엇이 들어있나요? 그들 사이에 어떤 차이가 있나요?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [16]:
response

ChatCompletion(id='chatcmpl-A6Cidhuqi39lc7RDeMDV1hiBS0MYf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='첫 번째 이미지와 두 번째 이미지 모두 나무로 만든 길이 있는 평야를 보여줍니다. 하지만 두 이미지 사이에는 다음과 같은 차이가 있습니다:\n\n1. **첫 번째 이미지**:\n   - 배경에 푸른 하늘과 흰 구름이 보이며, 매우 맑고 밝은 날의 분위기를 담고 있습니다.\n   - 경로 양 옆에 푸른 풀이 자라고 있어 자연의 푸르름이 돋보입니다.\n   - 넓은 초원이나 습지 같은 느낌을 주며, 주변에 나무와 숲도 일부 포함되어 있습니다.\n\n2. **두 번째 이미지**:\n   - 해질 무렵의 황혼을 배경으로 하고 있습니다. 하늘은 어두운 색조를 띠며 태양이 지고 있는 모습입니다.\n   - 경로 양 옆에 논이나 농작물 같은 형태의 식물이 자라고 있습니다.\n   - 멀리 보이는 산들은 자연 경관을 더욱 풍부하게 만듭니다.\n\n두 이미지 모두 자연 속의 길을 담고 있지만, 첫 번째 이미지는 밝고 푸르른 날씨가 강조되는 반면, 두 번째 이미지는 해질 무렵의 따뜻한 색감을 강조하고 있습니다.', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1726042223, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_992d1ea92d', usage=CompletionUsage(completion_tokens=271, prompt_tokens=1556, total_tokens=1827))

In [17]:
print(response.choices[0].message.content)

첫 번째 이미지와 두 번째 이미지 모두 나무로 만든 길이 있는 평야를 보여줍니다. 하지만 두 이미지 사이에는 다음과 같은 차이가 있습니다:

1. **첫 번째 이미지**:
   - 배경에 푸른 하늘과 흰 구름이 보이며, 매우 맑고 밝은 날의 분위기를 담고 있습니다.
   - 경로 양 옆에 푸른 풀이 자라고 있어 자연의 푸르름이 돋보입니다.
   - 넓은 초원이나 습지 같은 느낌을 주며, 주변에 나무와 숲도 일부 포함되어 있습니다.

2. **두 번째 이미지**:
   - 해질 무렵의 황혼을 배경으로 하고 있습니다. 하늘은 어두운 색조를 띠며 태양이 지고 있는 모습입니다.
   - 경로 양 옆에 논이나 농작물 같은 형태의 식물이 자라고 있습니다.
   - 멀리 보이는 산들은 자연 경관을 더욱 풍부하게 만듭니다.

두 이미지 모두 자연 속의 길을 담고 있지만, 첫 번째 이미지는 밝고 푸르른 날씨가 강조되는 반면, 두 번째 이미지는 해질 무렵의 따뜻한 색감을 강조하고 있습니다.
